In [1]:
import os
import boto3

In [2]:
import psycopg2

In [3]:
import requests
from pyspark.sql import SparkSession
from botocore.exceptions import ClientError

In [4]:
from airflow_client.client.api import dag_run_api
from airflow_client.client import Configuration, ApiClient

In [5]:
from airflow_client.client.model.dag_run import DAGRun

In [4]:
# 1. Inicializa Spark
spark = SparkSession.builder \
    .appName("teste") \
    .getOrCreate()

In [5]:
# 2. Conexão com MinIO
minio_client = boto3.client(
    's3',
    endpoint_url='http://datalake:9000',
    aws_access_key_id='admin',
    aws_secret_access_key='admin123'
)

bucket_name = 'meu-bucket-teste'

In [6]:
# 3. Criação de bucket no MinIO
try:
    minio_client.create_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' criado com sucesso.")
except ClientError as e:
    if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
        print(f"Bucket '{bucket_name}' já existe.")
    else:
        raise

Bucket 'meu-bucket-teste' já existe.


In [7]:
# 4. Upload de arquivos no MinIO
with open('/tmp/exemplo.txt', 'w') as f:
    f.write('Arquivo de teste para MinIO.')

minio_client.upload_file('/tmp/exemplo.txt', bucket_name, 'exemplo.txt')
print("Arquivo 'exemplo.txt' enviado ao MinIO.")

Arquivo 'exemplo.txt' enviado ao MinIO.


In [ ]:
# 5. Leitura de dados via Spark de um PostgreSQL
jdbc_url = "jdbc:postgresql://postgres:5432/meu_banco"
properties = {
    "user": "meu_usuario",
    "password": "minha_senha",
    "driver": "org.postgresql.Driver"
}

df = spark.read.jdbc(url=jdbc_url, table="minha_tabela", properties=properties)
df.show()

In [ ]:
# 6. Conexão com Airflow via REST API
AIRFLOW_API = 'http://airflow-apiserver:8080/api/v1'
DAG_ID = 'exemplo_dag'


def trigger_dag(dag_id):
    response = requests.post(
        f"{AIRFLOW_API}/dags/{dag_id}/dagRuns",
        auth=('airflow', 'airflow'),  # usuário e senha padrão
        json={"conf": {}}
    )
    if response.status_code == 200:
        print(f"DAG '{dag_id}' disparada com sucesso.")
    else:
        print(f"Falha ao disparar DAG: {response.status_code}, {response.text}")


trigger_dag(DAG_ID)

In [7]:
config = Configuration(
    host="http://localhost:8080/api/v2/version",
    username="air",
    password="flow"
)


with ApiClient(config) as api_client:
    api_instance = dag_run_api.DAGRunApi(api_client)
    dag_id = "meu_dag"

    dag_run = DAGRun(conf={})
    response = api_instance.post_dag_run(dag_id, dag_run)
    print(f"DAG triggered: {response.dag_run_id}")

MaxRetryError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /api/v2/version/dags/meu_dag/dagRuns (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8439399ed0>: Failed to establish a new connection: [Errno 111] Connection refused'))